<a href="https://colab.research.google.com/github/anmolag190153/BCS_summer_project_SER/blob/main/Speech_emotion_recognition_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import CSVLogger

In [2]:
sr=22100

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/My Drive/RAVDESS_dataset'
k = 0
x_data = np.ndarray(shape = (24 * 60, 120000))
y_data = np.ndarray(shape = (24 * 60, 1))
for i,actor in enumerate(os.listdir(path)):
    path1 = os.path.join(path,actor)
    for j, file in enumerate(os.listdir(path1)):
        b = file[18:20]
        y_data[k,0]= int(b) % 2
        temp_path = os.path.join(path1, file)
        # print(temp_path)
        var, _ = librosa.load(temp_path)
        x_data[k, :var.shape[0]] = var[:]
        x_data[k,var.shape[0]:] = 0
        k = k + 1

In [5]:
def build_fc_model(input_shape = (180, 1), num_class = 2):
    

    model=tf.keras.models.Sequential([tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu', input_shape = input_shape, kernel_regularizer = tf.keras.regularizers.l2(1e-2)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(1e-3)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(128, activation = 'relu'),
                                    tf.keras.layers.Dense(64, activation = 'relu'),
                                    tf.keras.layers.Dense(num_class, activation = 'softmax')
                                    ])

    return model

In [6]:
def extract_feature(file):
    result = np.array([])
    mfcc = np.mean(librosa.feature.mfcc(file, sr = sr, n_mfcc = 40).T, axis = 0)
    result = np.hstack((result, mfcc))
    chroma = np.mean(librosa.feature.chroma_stft(file, sr=sr).T, axis = 0)
    result = np.hstack((result, chroma))
    mel = np.mean(librosa.feature.melspectrogram(file, sr=sr).T, axis = 0)
    result = np.hstack((result, mel))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(file, sr = sr).T, axis = 0)
    result=np.hstack((result, spectral_rolloff))
    return result

In [7]:
x=[]
for i in range(24*60):
    feature = extract_feature(x_data[i,:])
    x.append(feature)
x_train, x_test, y_train, y_test = train_test_split(np.array(x), y_data, test_size = 0.2, random_state = 9)

In [8]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [11]:
model = build_fc_model()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
csv_logger = CSVLogger('model_history_gender.log', separator = ',', append = False)
history = model.fit(x_train, y_train, epochs = 500, batch_size = 256, validation_data = (x_test, y_test), callbacks = [csv_logger])

Epoch 1/500
5/5 [==============================] - 1s 62ms/step - loss: 3.5846 - accuracy: 0.7639 - val_loss: 3.1596 - val_accuracy: 0.9028
Epoch 2/500
5/5 [==============================] - 0s 26ms/step - loss: 2.9860 - accuracy: 0.9349 - val_loss: 2.8019 - val_accuracy: 0.9479
Epoch 3/500
5/5 [==============================] - 0s 30ms/step - loss: 2.6394 - accuracy: 0.9688 - val_loss: 2.4495 - val_accuracy: 0.9688
Epoch 4/500
5/5 [==============================] - 0s 27ms/step - loss: 2.3385 - accuracy: 0.9835 - val_loss: 2.1745 - val_accuracy: 0.9896
Epoch 5/500
5/5 [==============================] - 0s 27ms/step - loss: 2.0765 - accuracy: 0.9887 - val_loss: 1.9294 - val_accuracy: 0.9896
Epoch 6/500
5/5 [==============================] - 0s 29ms/step - loss: 1.8462 - accuracy: 0.9913 - val_loss: 1.7013 - val_accuracy: 1.0000
Epoch 7/500
5/5 [==============================] - 0s 26ms/step - loss: 1.6304 - accuracy: 0.9957 - val_loss: 1.5033 - val_accuracy: 1.0000
Epoch 8/500
5/5 [===

In [12]:
model.save('Gender_classification_mlp_model_Team-4')

INFO:tensorflow:Assets written to: Gender_classification_mlp_model_Team-4/assets


In [13]:
result = model.evaluate(x_test, y_test)
print('The accuracy obtained on validation is ', result[1] * 100, '%')

9/9 [==============================] - 0s 3ms/step - loss: 0.0695 - accuracy: 0.9931
The accuracy obtained on validation is  99.30555820465088 %
